In [1]:
import torch

# Load the model
checkpoint = torch.load('/home/cadt-02/Object-Detection/ros2_ws/model_- 31 january 2024 11_41.pt')
model = checkpoint['model']
model.eval()

KeyboardInterrupt: 

In [ ]:


try:
    # Convert the model's parameters to float data type
    model.float()
    
    input_tensor = torch.randn(1, 3, 640, 640).to(next(model.parameters()).device).type(model.parameters().__next__().dtype)
    torch.onnx.export(model, input_tensor, "model.onnx")
except RuntimeError as e:
    print(f"An error occurred: {str(e)}")


/home/cadt-02/.local/lib/python3.8/site-packages/ultralytics/nn/modules/head.py:53: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if self.dynamic or self.shape != shape:
/home/cadt-02/.local/lib/python3.8/site-packages/ultralytics/utils/tal.py:299: TracerWarning: Iterating over a tensor might cause the trace to be incorrect. Passing a tensor of different shape won't change the number of iterations executed (and might lead to errors or silently give incorrect results).
  for i, stride in enumerate(strides):


In [ ]:
import os
import tensorrt as trt
import pycuda.driver as cuda
import pycuda.autoinit

TRT_LOGGER = trt.Logger(trt.Logger.WARNING)

def get_engine(onnx_file_path, engine_file_path):
    # Check if the engine exists
    if os.path.exists(engine_file_path):
        # If it does, load the engine and return
        with open(engine_file_path, "rb") as f, trt.Runtime(TRT_LOGGER) as runtime:
            return runtime.deserialize_cuda_engine(f.read())
    else:
        # If it doesn't, we need to build the engine from the ONNX model.
        with trt.Builder(TRT_LOGGER) as builder, builder.create_network(1) as network, trt.OnnxParser(network, TRT_LOGGER) as parser:
            config = builder.create_builder_config()
            config.max_workspace_size = 1 << 28  # 256MiB
            builder.max_batch_size = 1
            # Parse the ONNX model
            with open(onnx_file_path, 'rb') as model:
                if not parser.parse(model.read()):
                    print('ERROR: Failed to parse the ONNX file.')
                    for error in range(parser.num_errors):
                        print(parser.get_error(error))
                    return None
            # Build and return an engine.
            engine = builder.build_engine(network, config)
            with open(engine_file_path, "wb") as f:
                f.write(engine.serialize())
            return engine

# Specify the paths to the ONNX model and the engine
onnx_file_path = 'model.onnx'
engine_file_path = 'model.trt'

# Get the engine
engine = get_engine(onnx_file_path, engine_file_path)

[03/06/2024-11:14:57] [TRT] [W] Using an engine plan file across different models of devices is not recommended and is likely to affect performance or even cause errors.


In [1]:
import tensorrt as trt
import pycuda.driver as cuda
import pycuda.autoinit
import numpy as np

# Load your TensorRT model
with open("model.trt", "rb") as f, trt.Runtime(trt.Logger()) as runtime:
    engine = runtime.deserialize_cuda_engine(f.read())

# Create an execution context
context = engine.create_execution_context()

# Allocate buffers
inputs = []
outputs = []
bindings = []
stream = cuda.Stream()
for binding in engine:
    size = trt.volume(engine.get_binding_shape(binding)) * engine.max_batch_size
    dtype = trt.nptype(engine.get_binding_dtype(binding))
    # Allocate host and device buffers
    host_mem = cuda.pagelocked_empty(size, dtype)
    device_mem = cuda.mem_alloc(host_mem.nbytes)
    # Append the device buffer to device bindings.
    bindings.append(int(device_mem))
    # Append to the appropriate list.
    if engine.binding_is_input(binding):
        inputs.append((host_mem, device_mem))
    else:
        outputs.append((host_mem, device_mem))

# Assume input_data is your input data
input_data = np.random.random_sample(size)
input_data = np.reshape(input_data, inputs[0][0].shape)
np.copyto(inputs[0][0], input_data.ravel())

# Transfer input data to the GPU.
cuda.memcpy_htod_async(inputs[0][1], inputs[0][0], stream)

# Run inference
context.execute_async(bindings=bindings, stream_handle=stream.handle)

# Transfer predictions back from the GPU.
cuda.memcpy_dtoh_async(outputs[0][0], outputs[0][1], stream)

# Synchronize the stream
stream.synchronize()

# Our model outputs are now in the outputs[0][0] buffer.
predictions = outputs[0][0]
print(predictions)

[03/07/2024-16:04:31] [TRT] [W] Using an engine plan file across different models of devices is not recommended and is likely to affect performance or even cause errors.


/tmp/ipykernel_87647/4022393972.py:19: DeprecationWarning: Use get_tensor_shape instead.
  size = trt.volume(engine.get_binding_shape(binding)) * engine.max_batch_size
/tmp/ipykernel_87647/4022393972.py:19: DeprecationWarning: Use network created with NetworkDefinitionCreationFlag::EXPLICIT_BATCH flag instead.
  size = trt.volume(engine.get_binding_shape(binding)) * engine.max_batch_size
/tmp/ipykernel_87647/4022393972.py:20: DeprecationWarning: Use get_tensor_dtype instead.
  dtype = trt.nptype(engine.get_binding_dtype(binding))
/tmp/ipykernel_87647/4022393972.py:27: DeprecationWarning: Use get_tensor_mode instead.
  if engine.binding_is_input(binding):


[03/07/2024-16:04:33] [TRT] [W] The getMaxBatchSize() function should not be used with an engine built from a network created with NetworkDefinitionCreationFlag::kEXPLICIT_BATCH flag. This function will always return 1.
[03/07/2024-16:04:33] [TRT] [W] The getMaxBatchSize() function should not be used with an engine built from a network created with NetworkDefinitionCreationFlag::kEXPLICIT_BATCH flag. This function will always return 1.
[03/07/2024-16:04:33] [TRT] [W] The getMaxBatchSize() function should not be used with an engine built from a network created with NetworkDefinitionCreationFlag::kEXPLICIT_BATCH flag. This function will always return 1.
[03/07/2024-16:04:33] [TRT] [W] The getMaxBatchSize() function should not be used with an engine built from a network created with NetworkDefinitionCreationFlag::kEXPLICIT_BATCH flag. This function will always return 1.
[03/07/2024-16:04:33] [TRT] [W] The getMaxBatchSize() function should not be used with an engine built from a network cr

ValueError: cannot reshape array of size 58800 into shape (1228800,)